# Unsupervised Anomaly Detection: Schools Performance 2016-17
## Objective
The focus of this notebook/project is to investigate the use of **unsupervised anomaly detection** methods on **Schools Performance 2016-17** data. We are particularly interested in spotting anomalous pupil destination data. 

## Definitions
The following terms used in this project are defined below:
+ **Unsupervised data:** When your data is not labelled. *e.g. If you have income data but you do not know if this data is correct*
+ **Anomaly data:** Unusual and typically wrong observations/data points in your data.
+ **Unsupervised anomaly detection:** Spotting anomaly observations in your data without any labelled data to tell you whether your predictions are correct.

## Data
The data for this project comes from the [Schools Performance 2016-17](https://www.compare-school-performance.service.gov.uk/). The data is here very rich and detailed. For the sake of demonstration and simplification., we will only consider a subset of this information, defined below:
+ **URN** | Unique reference number for schools
+ **LEA** | Local authority, as a code
+ **ESTAB** | Establishment number
    - *Note that combining the LA and ESTAB fields to give LAESTAB will give us another unique identifer for schools, akin to our URN field*
+ **SCHNAME** | School name
+ **NFTYPE** | School type
    - AC = Sponsored academy
    - ACC = Academy converter – mainstream
    - AC1619 = Academy 16-19 sponsor led
    - ACC1619 = Academy 16-19 converter
    - ACCS = Academy converter - special school
    - ACS = Sponsored special academy, CTC=City technology college
    - CY = Community school
    - CYS = Community special school
    - F = Free school – mainstream
    - FESI = Further Education Sector Institution
    - FD = Foundation school
    - FDS = Foundation special school
    - FS = Free school – special
    - FSS = Studio school
    - FUTC = UTC (university technical college)
    - F1619 = Free school - 16-19
    - IND = Independent school
    - INDSPEC = Independent special school
    - MODFC = College funded by Ministry of Defence
    - NMSS = Non-maintained special school
    - VA = Voluntary aided school,
    - VC = Voluntary controlled school
+ **OVERALL_DESTPER** | Percentage of pupils who have been in a sustained education or employment destination for the first two terms, October 2015 to March 2016. 
+ **PTEALGRP2** | Percentage of eligible pupils with English-as-(an)-language (EAL)
+ **PTMOBN** | Percentage of pupils classified as non-mobile
+ **PTRWM_EXP** | Percentage of pupils reaching the expected stnadard in reading, writing and maths
+ **PSENELST** | Percentage of eligible pupils with special-education-needs (SEN)
+ **PTFSM6CLA1A_16** | Percentage of KS2 disdvantaged pupils
+ **PNUMFSM** | Percentage of pupils for free-school-meals

The anomalous pupil destination data that we are interested in spotting is therefore found within the **OVERALL_DESTPER** field. Note that this information is *provisional* so it is **unlabelled**.

## Set-up
Need to set-up our Jupyter notebook so that it has the required libraries and an environment is set-up so that when sharing this noteboook, others can use the same environment as we did here.

By default, the use of `!conda install` will install the package to the environment location that we are running the kernel in. This would be the environment from which we started Jupyter notebook from, but we can check this by looking at some of the system variables from the sys module.

In [2]:
import sys
sys.executable

'C:\\Users\\a_vis\\Anaconda3\\python.exe'

In [1]:
# Allows multiple outputs to show from same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Show the two dataframes side-by-side by creating a function
from IPython.display import display_html
def display_side(*args):
    html_str = ''
    for df in args:
        html_str += df.to_html()
    display_html(html_str.replace('table', 'table style = "display:inline"'),raw = True)


In [27]:
# Use pip freeze to examine installed packages and versions within our Jupyter session
!pip freeze

alabaster==0.7.9
anaconda-clean==1.0
anaconda-client==1.6.3
anaconda-navigator==1.6.4
anaconda-project==0.6.0
argcomplete==1.0.0
astroid==1.4.7
astropy==1.2.1
Babel==2.3.4
backports.shutil-get-terminal-size==1.0.0
beautifulsoup4==4.5.1
bitarray==0.8.1
blaze==0.10.1
bokeh==0.12.2
boto==2.42.0
Bottleneck==1.1.0
cffi==1.7.0
chardet==3.0.4
chest==0.2.3
click==6.6
cloudpickle==0.2.1
clyent==1.2.2
colorama==0.3.7
comtypes==1.1.2
conda==4.5.0
conda-build==2.0.2
configobj==5.0.6
contextlib2==0.5.3
cryptography==1.5
cycler==0.10.0
Cython==0.24.1
cytoolz==0.8.0
dask==0.11.0
datashape==0.5.2
decorator==4.0.10
dill==0.2.5
docutils==0.12
dplython==0.0.7
dynd===c328ab7
et-xmlfile==1.0.1
fastcache==1.0.2
filelock==2.0.6
Flask==0.11.1
Flask-Cors==2.1.2
gevent==1.1.2
greenlet==0.4.10
h5py==2.7.1
HeapDict==1.0.0
idna==2.1
imagesize==0.7.1
ipykernel==4.5.0
ipython==5.1.0
ipython-genutils==0.1.0
ipywidgets==5.2.2
itsdangerous==0.24
jdcal==1.2
jedi==0.9.0
Jinja2==2.8
jsonschema==2.5.1
jupyter==1.0.0
jupyte

You are using pip version 8.1.2, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
C:\Users\a_vis\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:132: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)
C:\Users\a_vis\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:132: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  return process_handler(cmd, _system_body)
C:\Users\a_vis\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:132: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  return process_handler(cmd, _system_body)


In [5]:
# Install libraries not already above
!conda install seaborn -y
!pip install dplython

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\a_vis\Anaconda3

  added / updated specs: 
    - seaborn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    seaborn-0.8.1              |   py35hc73483e_0         338 KB
    openssl-1.0.2o             |       h8ea7d77_0         5.4 MB
    ------------------------------------------------------------
                                           Total:         5.7 MB

The following NEW packages will be INSTALLED:

    seaborn:         0.8.1-py35hc73483e_0            

The following packages will be UPDATED:

    ca-certificates: 2018.1.18-0          conda-forge --> 2018.03.07-0     
    openssl:         1.0.2n-vc14_0        conda-forge [vc14] --> 1.0.2o-h8ea7d77_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done



seaborn 0.8.1:            |   0% 
seaborn 0.8.1: 3          |   4% 
seaborn 0.8.1: #######5   |  75% 
seaborn 0.8.1: #########3 |  93% 
seaborn 0.8.1: ########## | 100% 

openssl 1.0.2o:            |   0% 
openssl 1.0.2o: 1          |   2% 
openssl 1.0.2o: 6          |   6% 
openssl 1.0.2o: #1         |  11% 
openssl 1.0.2o: #7         |  18% 
openssl 1.0.2o: ##2        |  23% 
openssl 1.0.2o: ##7        |  28% 
openssl 1.0.2o: ###3       |  34% 
openssl 1.0.2o: ###8       |  39% 
openssl 1.0.2o: ####4      |  45% 
openssl 1.0.2o: #####      |  50% 
openssl 1.0.2o: #####7     |  57% 
openssl 1.0.2o: ######3    |  63% 
openssl 1.0.2o: ######9    |  70% 
openssl 1.0.2o: #######5   |  76% 
openssl 1.0.2o: ########   |  80% 
openssl 1.0.2o: ########4  |  84% 
openssl 1.0.2o: ########8  |  88% 
openssl 1.0.2o: #########1 |  92% 
openssl 1.0.2o: #########4 |  95% 
openssl 1.0.2o: #########7 |  98% 
openssl 1.0.2o: ########## | 100% 


Having downloaded all our required libraries, and set-up our environment as we wanted, we will save this environment so others taking this Jupyter notebook can use the same environment as us in order to replicate our results, and not go through the same hassle of going through the parts above this comment block. They will download this environment within their own Anaconda Navigator.

Creating this environemnt, can upload it to my Anaconda Cloud account, which would allow other users to download and install it as a new environment within their own Anaconda Navigator.

In [1]:
# Export our environment, "NewEnv" and save it as "anomaly-detection.yml"
!conda env export -n NewEnv -f anomaly-detection.yml

# Data Load, Consolidation and Wrangling
In this section, we will load in our data, join our different data sources together, and manipulate our joined dataframe so that it is in a format for us to use various **anomaly detection** algorithms on.

In [11]:
# Check working directory to ensure user notebook is easily transferable
import os
os.getcwd()

'C:\\Users\\a_vis\\Documents\\Data Science\\Projects\\Anomaly Detection\\local_master.git'

**Dplython** is a useful layer on top of Python's **pandas** library that maps R's *dplyr* functionality to a library in Python. More information can be found at this [site](https://pythonhosted.org/dplython/).
*Note: The `sift()` function is the **Dplython** equivalent of **dplyr's** `filter()` and `@DelayFunction` is used to pass dataframes/columns into functions.*

In [225]:
# Import required libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from dplython import (
    DplyFrame, 
    X,
    select, 
    arrange, 
    mutate, 
    group_by, 
    summarize, 
    sift, 
    DelayFunction
)

# Display plots in Jupyter notebooks
% matplotlib inline

# Set random seed so we can reproduce our outputs
rng = np.random.RandomState(123)

# Load data
data_provisional_destination = pd.read_csv('Data 2016-2017/england_pupdestprovisional.csv', encoding = 'UTF-8')
data_final_ks2 = pd.read_csv('Data 2016-2017/england_ks2final.csv', encoding = 'UTF-8')

C:\Users\a_vis\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1,2,3,4,12,14,16,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Cleaning Data
Need to clean data first by following the below steps:
1. Remove uneccessary columns
1. Remove **'NaN'** entries in our unique identifier (*this corresponds to non-provider level data, such as being at the local authority or national level; RECTYPE = 3 for data_provisional_destination and RECTYPE >= 3 for data_final_ks2*) 
1. Convert remaining columns to right datatype 


In [226]:
# Take only relevant columns
data_provisional_destination = data_provisional_destination[['URN', 'SCHNAME',
                                                            'NFTYPE', 'OVERALL_DESTPER']]
data_final_ks2 = data_final_ks2[['URN', 'SCHNAME', 'NFTYPE',
                                'PTEALGRP2', 'PTMOBN', 'PTRWM_EXP',
                                'PSENELST', 'PTFSM6CLA1A_16']]

# View the two dataframes
display_side(data_provisional_destination.head(), data_final_ks2.head())

URN 
 SCHNAME 
 NFTYPE 
 OVERALL_DESTPER 
 
 
 
 
 0 
 100049.0 
 Haverstock School 
 CY 
 88% 
 
 
 1 
 100050.0 
 Parliament Hill School 
 CY 
 92% 
 
 
 2 
 100051.0 
 Regent High School 
 CY 
 91% 
 
 
 3 
 100052.0 
 Hampstead School 
 CY 
 94% 
 
 
 4 
 100053.0 
 Acland Burghley School 
 CY 
 88% 
 
 
 
 
 
 
 URN 
 SCHNAME 
 NFTYPE 
 PTEALGRP2 
 PTMOBN 
 PTRWM_EXP 
 PSENELST 
 PTFSM6CLA1A_16 
 
 
 
 
 0 
 110705 
 The Duke of Bedford Primary School 
 CY 
 4% 
 92% 
 50% 
 0% 
 22% 
 
 
 1 
 117736 
 East Halton Primary School 
 CY 
 0% 
 100% 
 33% 
 0% 
 SUPP 
 
 
 2 
 122638 
 East Markham Primary School 
 CY 
 0% 
 95% 
 67% 
 0% 
 0% 
 
 
 3 
 141550 
 Easton Primary School 
 ACC 
 0% 
 75% 
 88% 
 0% 
 0% 
 
 
 4 
 142999 
 Eastwick Junior School 
 ACC 
 5% 
 98% 
 64% 
 0% 
 13%

In [227]:
# Obtain datatypes
data_provisional_destination.dtypes
data_final_ks2.dtypes

URN                float64
SCHNAME             object
NFTYPE              object
OVERALL_DESTPER     object
dtype: object

URN               object
SCHNAME           object
NFTYPE            object
PTEALGRP2         object
PTMOBN            object
PTRWM_EXP         object
PSENELST          object
PTFSM6CLA1A_16    object
dtype: object

In [228]:
# Check for NaNs in unique identifier field
print(data_final_ks2[data_final_ks2['URN'].isnull()])
# No NaNs, potentially because URN is being read as object/string datatype
# so blanks entries in data which we expect to be NaNs are read as ""
data_final_ks2.loc[data_final_ks2['URN'] == ' '].head(10)
# Replace these empty cells with 'NaN
data_final_ks2['URN'] = data_final_ks2['URN'].replace(r'\s+', np.nan, regex=True)
# Check this worked
data_final_ks2[data_final_ks2['URN'].isnull()].head(10)

Empty DataFrame
Columns: [URN, SCHNAME, NFTYPE, PTEALGRP2, PTMOBN, PTRWM_EXP, PSENELST, PTFSM6CLA1A_16]
Index: []


,URN,SCHNAME,NFTYPE,PTEALGRP2,PTMOBN,PTRWM_EXP,PSENELST,PTFSM6CLA1A_16
2560,,Oldham,,36%,95%,57%,1%,42%
2561,,Kent,,10%,93%,65%,0%,28%
2584,,Essex,,6%,93%,63%,1%,26%
2585,,Telford and Wrekin,,10%,92%,61%,1%,36%
2586,,Norfolk,,9%,92%,57%,1%,29%
2587,,Northamptonshire,,14%,93%,57%,1%,27%
2735,,Bury,,18%,94%,63%,1%,30%
2736,,Poole,,9%,92%,59%,1%,24%
2737,,West Berkshire,,7%,94%,62%,1%,18%
2738,,Cheshire East,,5%,93%,64%,1%,21%


,URN,SCHNAME,NFTYPE,PTEALGRP2,PTMOBN,PTRWM_EXP,PSENELST,PTFSM6CLA1A_16
2560,NaN,Oldham,,36%,95%,57%,1%,42%
2561,NaN,Kent,,10%,93%,65%,0%,28%
2584,NaN,Essex,,6%,93%,63%,1%,26%
2585,NaN,Telford and Wrekin,,10%,92%,61%,1%,36%
2586,NaN,Norfolk,,9%,92%,57%,1%,29%
2587,NaN,Northamptonshire,,14%,93%,57%,1%,27%
2735,NaN,Bury,,18%,94%,63%,1%,30%
2736,NaN,Poole,,9%,92%,59%,1%,24%
2737,NaN,West Berkshire,,7%,94%,62%,1%,18%
2738,NaN,Cheshire East,,5%,93%,64%,1%,21%


In [230]:
# Check length of dataframes before removing NaNs
len(data_provisional_destination)

# Remove rows at LA and NAT level/where 'URN' is NaN
data_provisional_destination = data_provisional_destination[data_provisional_destination['URN'].notnull()]
len(data_provisional_destination)

3843

3691

In [231]:
# Check length of dataframes before removing NaNs
len(data_final_ks2)

# Remove rows at LA and NAT level - where 'URN' is NaN
# Note, process different here because URN is a 'object'/string datatype
data_final_ks2['URN'] = pd.to_numeric(data_final_ks2['URN'], errors='coerce')
data_final_ks2 = data_final_ks2.dropna(subset=['URN'])
len(data_final_ks2)

16467

16313

In [234]:
data_provisional_destination.dtypes
data_final_ks2.dtypes

URN                float64
SCHNAME             object
NFTYPE              object
OVERALL_DESTPER     object
dtype: object

URN               float64
SCHNAME            object
NFTYPE             object
PTEALGRP2          object
PTMOBN             object
PTRWM_EXP          object
PSENELST           object
PTFSM6CLA1A_16     object
dtype: object

# CONTINUE HERE
To do:
1. Deal with SUPP values
1. Convert % to floats/decimals. Useful link [here](https://stackoverflow.com/questions/25669588/convert-percent-string-to-float-in-pandas-read-csv)

In [233]:
# Convert to right datatypes
data_provisional_destination['URN'] = pd.to_numeric(data_provisional_destination['URN'])
data_final_ks2['URN'] = pd.to_numeric(data_final_ks2['URN'])

In [117]:
# Join two dataframes on URN
data_final = data_provisional_destination.merge(right = data_final_ks2, on = 'URN', how = 'left',
                                               suffixes = ['', '_temp'])
data_final.tail(10)

,﻿RECTYPE,LEA,ESTAB,URN,ICLOSE,SCHNAME,NFTYPE,COHORT,OVERALL_DEST,APPREN,...,READPROG_LOWER_16,READPROG_UPPER_16,WRITPROG_16,WRITPROG_LOWER_16,WRITPROG_UPPER_16,MATPROG_16,MATPROG_LOWER_16,MATPROG_UPPER_16,READ_AVERAGE_16,MAT_AVERAGE_16
3681,2,938.0,7003.0,126154.0,0.0,Muntham House School,NMSS,8,SUPP,SUPP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3682,2,938.0,7004.0,126155.0,0.0,St Anthony's School,CYS,23,22,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3683,2,938.0,7005.0,126156.0,0.0,"Littlegreen School, Compton",CYS,5,SUPP,SUPP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3684,2,938.0,7006.0,126157.0,0.0,Manor Green College,CYS,28,26,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3685,2,938.0,7009.0,126160.0,0.0,"Queen Elizabeth II Silver Jubilee School, Horsham",CYS,8,SUPP,SUPP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3686,2,938.0,7010.0,126161.0,0.0,Oak Grove College,CYS,26,24,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3687,2,938.0,7012.0,126163.0,0.0,"Fordwater School, Chichester",CYS,6,SUPP,SUPP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3688,2,938.0,7013.0,136114.0,0.0,Woodlands Meed,FDS,30,28,SUPP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3689,2,938.0,7022.0,126170.0,0.0,"Cornfield School, Littlehampton",CYS,9,SUPP,SUPP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3690,2,938.0,7023.0,135814.0,0.0,Ingfield Manor School,NMSS,2,SUPP,SUPP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
# Check number of rows in data_final is same as data_provsional_destination
data_final.index
data_provisional_destination.index
data_final_ks2.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            3833, 3834, 3835, 3836, 3837, 3838, 3839, 3840, 3841, 3842],
           dtype='int64', length=3843)

RangeIndex(start=0, stop=3843, step=1)

RangeIndex(start=0, stop=16467, step=1)

In [57]:
# Concatenate 'LES' and 'ESTAB' fields to give 'LAESTAB', a unique identifier
data_final_temp = DplyFrame(data_final) 

In [56]:
(
    data_final_temp = data_final_temp 
    >> mutate(LAESTAB = 'LEA' + 'ESTAB')
)

SyntaxError: invalid syntax (<ipython-input-56-ef725b91f7e9>, line 2)

In [29]:
#data_final['LEA'] + data_final['ESTAB']

SyntaxError: invalid syntax (<ipython-input-29-54b88196c8e5>, line 3)

In [58]:
data_final_temp.head(10)

,URN,LEA,ESTAB,SCHNAME,NFTYPE,OVERALL_DESTPER,LEA_temp,ESTAB_temp,SCHNAME_temp,NFTYPE_temp,PTEALGRP2,PTMOBN,PTRWM_EXP,PSENELST,PTFSM6CLA1A_16
0,100049.0,202.0,4104.0,Haverstock School,CY,88%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100050.0,202.0,4166.0,Parliament Hill School,CY,92%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100051.0,202.0,4196.0,Regent High School,CY,91%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100052.0,202.0,4275.0,Hampstead School,CY,94%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100053.0,202.0,4285.0,Acland Burghley School,CY,88%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,100054.0,202.0,4611.0,The Camden School for Girls,VA,94%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,100055.0,202.0,4652.0,Maria Fidelis Roman Catholic Convent School FCJ,VA,92%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,100056.0,202.0,4688.0,William Ellis School,VA,97%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,100059.0,202.0,5401.0,La Sainte Union Catholic Secondary School,VA,97%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,100092.0,202.0,7137.0,Camden Centre for Learning (CCfL) Special School,CYS,SUPP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
